In [1]:
import pandas as pd
import re

Importer les données disponibles sur le site https://www.data.gouv.fr. \
Ce jeu de données présente les profils horaires des validations des voyageurs par jour type et par arrêt sur le réseau ferré au second semestre 2022

In [2]:
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = '/workspaces/metro-JO/Données importées/2022_S2_PROFIL_FER.csv'
données_validation = pd.read_csv(url, delimiter=';')

print("Tableau importé : profils horaires des validations des voyageurs par jour type et par arrêt au second semestre 2022")
display(données_validation)

Tableau importé : profils horaires des validations des voyageurs par jour type et par arrêt au second semestre 2022


/tmp/ipykernel_33608/3496905205.py:3: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  données_validation = pd.read_csv(url, delimiter=';')


,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,lda,CAT_JOUR,TRNC_HORR_60,pourc_validations
0,100,110,1,PORTE MAILLOT,71379,DIJFP,0H-1H,"2,18"
1,100,110,1,PORTE MAILLOT,71379,DIJFP,10H-11H,"5,03"
2,100,110,1,PORTE MAILLOT,71379,DIJFP,11H-12H,"5,72"
3,100,110,1,PORTE MAILLOT,71379,DIJFP,12H-13H,"6,10"
4,100,110,1,PORTE MAILLOT,71379,DIJFP,13H-14H,"5,87"
...,...,...,...,...,...,...,...,...
85249,810,802,905,VILLEPINTE,73547,SAVS,5H-6H,"4,25"
85250,810,802,905,VILLEPINTE,73547,SAVS,6H-7H,"7,22"
85251,810,802,905,VILLEPINTE,73547,SAVS,7H-8H,"6,79"
85252,810,802,905,VILLEPINTE,73547,SAVS,8H-9H,"6,39"


Tableau avec les pourcentages de validations pour chaque arrêt et chaque tranche horaire

In [3]:
# Convertir la colonne 'pourc_validations' en float (remplacer la virgule par un point pour séparer les décimales)
données_validation['pourc_validations'] = données_validation['pourc_validations'].str.replace(',', '.').astype(float)

# Grouper par 'LIBELLE_ARRET' et 'TRNC_HORR_60', puis calculer la somme des pourc_validations pour chaque groupe
tableau_somme_pour_validations = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()

# Convertir les tranches horaires au format heure
tranche_horaire_to_hour = {
    '0H-1H': '00:00-01:00',
    '1H-2H': '01:00-02:00',
    '2H-3H': '02:00-03:00',
    '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00',
    '5H-6H': '05:00-06:00',
    '6H-7H': '06:00-07:00',
    '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00',
    '9H-10H': '09:00-10:00',
    '10H-11H': '10:00-11:00',
    '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00',
    '13H-14H': '13:00-14:00',
    '14H-15H': '14:00-15:00',
    '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00',
    '17H-18H': '17:00-18:00',
    '18H-19H': '18:00-19:00',
    '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00',
    '21H-22H': '21:00-22:00',
    '22H-23H': '22:00-23:00',
    '23H-0H': '23:00-00:00'
}
tableau_somme_pour_validations['TRNC_HORR_60'] = tableau_somme_pour_validations['TRNC_HORR_60'].map(tranche_horaire_to_hour)

print("Tableau avec la somme des pourcentages de validations pour chaque arrêt et chaque tranche horaire au second semestre 2022")
display(tableau_somme_pour_validations)


Tableau avec la somme des pourcentages de validations pour chaque arrêt et chaque tranche horaire au second semestre 2022


,LIBELLE_ARRET,TRNC_HORR_60,pourc_validations
0,ABBESSES,00:00-01:00,10.18
1,ABBESSES,10:00-11:00,19.26
2,ABBESSES,11:00-12:00,23.72
3,ABBESSES,12:00-13:00,27.00
4,ABBESSES,13:00-14:00,29.13
...,...,...,...
17610,YVRIS NOISY GD,05:00-06:00,10.08
17611,YVRIS NOISY GD,06:00-07:00,36.43
17612,YVRIS NOISY GD,07:00-08:00,69.33
17613,YVRIS NOISY GD,08:00-09:00,73.34


Tableau avec les pourcentages de validations pour chaque tranche horaire par arrêt 

In [4]:
# Regroupement des données en cas de doublons pour une paire 'LIBELLE_ARRET'-'TRNC_HORR_60'
grouped_data = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()

# Pivotement des données
pivot_table = grouped_data.pivot(index='LIBELLE_ARRET', columns='TRNC_HORR_60', values='pourc_validations').reset_index()

# Renommer les colonnes
pivot_table.columns.name = None  
pivot_table = pivot_table.rename(columns=lambda x: f"{x}" if x != 'LIBELLE_ARRET' else x)

# Sélectionner les colonnes correspondant aux tranches horaires
horaire_columns = pivot_table.columns[1:]  

# Fonction pour extraire l'heure de début de chaque tranche horaire
def extract_start_hour(column):
    match = re.match(r'(\d+)H-(\d+)H', column)
    if match:
        return int(match.group(1))
    return 0

# Trier les colonnes dans l'ordre croissant des heures de début des tranches
sorted_columns = sorted(horaire_columns, key=extract_start_hour)

# Réorganiser le tableau pivoté avec les colonnes dans l'ordre chronologique
new_column_order = ['LIBELLE_ARRET'] + sorted_columns
pivot_table = pivot_table[new_column_order]

# Supprimer la colonne 'ND' du DataFrame pivot_table_ordered
pivot_table.drop(columns=['ND'], inplace=True, errors='ignore')

pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.strip()  # Supprime les espaces en début et fin
pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.lower()  # Met tout en minuscules

# Convertir les tranches horaires au format heure
tranche_horaire_to_hour = {
    '0H-1H': '00:00-01:00',
    '1H-2H': '01:00-02:00',
    '2H-3H': '02:00-03:00',
    '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00',
    '5H-6H': '05:00-06:00',
    '6H-7H': '06:00-07:00',
    '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00',
    '9H-10H': '09:00-10:00',
    '10H-11H': '10:00-11:00',
    '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00',
    '13H-14H': '13:00-14:00',
    '14H-15H': '14:00-15:00',
    '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00',
    '17H-18H': '17:00-18:00',
    '18H-19H': '18:00-19:00',
    '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00',
    '21H-22H': '21:00-22:00',
    '22H-23H': '22:00-23:00',
    '23H-0H': '23:00-00:00'
}
pivot_table.rename(columns=tranche_horaire_to_hour, inplace=True)

# Affichage du tableau pivoté
print("Tableau pivoté avec les pourcentages de validations pour chaque tranche horaire par arrêt au second semestre 2022")
display(pivot_table)


Tableau pivoté avec les pourcentages de validations pour chaque tranche horaire par arrêt au second semestre 2022


,LIBELLE_ARRET,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,08:00-09:00,...,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00
0,abbesses,10.18,3.20,0.02,0.00,NaN,1.31,4.34,10.27,24.71,...,34.04,40.12,43.49,45.59,43.24,37.00,23.36,19.02,19.36,18.35
1,ablon,0.26,0.03,0.00,0.02,0.51,17.57,35.26,53.54,54.92,...,27.21,27.36,29.27,31.79,26.71,21.33,12.33,7.63,4.63,1.65
2,acheres grand,0.78,NaN,NaN,NaN,0.58,5.64,15.11,29.33,43.88,...,29.83,51.07,54.95,58.62,25.58,12.67,9.33,4.94,11.87,9.08
3,acheres ville,0.76,0.31,0.13,0.09,3.00,11.85,22.20,47.28,55.96,...,31.10,31.14,33.09,33.95,26.21,18.53,12.86,8.03,4.92,2.10
4,aeroport cdg1,0.76,0.14,0.01,0.04,5.06,9.12,12.09,16.26,17.69,...,39.21,39.22,41.98,46.46,39.65,33.54,30.27,21.77,20.00,8.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,vosves,NaN,NaN,NaN,NaN,NaN,0.53,40.41,95.90,33.36,...,46.84,26.49,50.68,29.68,24.99,21.65,24.82,14.16,0.75,NaN
752,vulaines sur se,0.11,NaN,NaN,NaN,NaN,10.82,23.52,50.78,45.09,...,34.67,32.19,46.84,36.68,26.39,14.70,11.62,6.93,3.46,0.05
753,wagram,3.63,1.43,0.10,0.00,0.01,1.86,4.51,12.16,26.16,...,34.50,37.50,39.59,47.13,47.83,40.00,22.20,10.86,9.79,7.89
754,yerres,0.73,0.11,0.01,0.05,1.88,14.18,32.07,60.03,61.84,...,27.72,26.68,27.94,29.82,24.98,17.29,10.77,5.26,3.83,1.91


Importer les données disponibles sur le site https://www.data.gouv.fr.\
Ce jeu de données présente le nombre de validations des voyageurs par jour par arrêt et par titre de transport sur le réseau ferré au second semestre 2022.

In [5]:
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url2 = '/workspaces/metro-JO/Données importées/2022_S2_NB_FER.csv'
données_validation2 = pd.read_csv(url2, delimiter=';')
print("Tableau importé : nombre de validations par jour par arrêt et par titre de transport au second semestre 2022")
display(données_validation2)


Tableau importé : nombre de validations par jour par arrêt et par titre de transport au second semestre 2022


,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,lda,CATEGORIE_TITRE,NB_VALD
0,01/07/2022,100,110,1,PORTE MAILLOT,71379,?,253
1,01/07/2022,100,110,1,PORTE MAILLOT,71379,AMETHYSTE,171
2,01/07/2022,100,110,1,PORTE MAILLOT,71379,AUTRE TITRE,1398
3,01/07/2022,100,110,1,PORTE MAILLOT,71379,FGT,247
4,01/07/2022,100,110,1,PORTE MAILLOT,71379,IMAGINE R,2510
...,...,...,...,...,...,...,...,...
1105942,31/12/2022,810,802,905,VILLEPINTE,73547,IMAGINE R,212
1105943,31/12/2022,810,802,905,VILLEPINTE,73547,NAVIGO,614
1105944,31/12/2022,810,802,905,VILLEPINTE,73547,NAVIGO JOUR,6
1105945,31/12/2022,810,802,905,VILLEPINTE,73547,NON DEFINI,6


In [6]:
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], errors='coerce', format="%d/%m/%Y")
donnees_mai = données_validation2.loc[(données_validation2['JOUR'].dt.month == 5)]
donnees_sum = donnees_mai.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Conversion de la colonne 'JOUR' en datetime
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrer les données pour les dates spécifiques : 24/07/2022 au 11/08/2022
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Grouper par 'JOUR', 'LIBELLE_ARRET' et 'CATEGORIE_TITRE' et calculer la somme des 'NB_VALD'
donnees_aggreg = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET', 'CATEGORIE_TITRE'])['NB_VALD'].sum().reset_index()

# Conversion de la colonne 'JOUR' en datetime si ce n'est pas déjà fait
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrage des dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Somme de NB_VALD par JOUR et par LIBELLE_ARRET
somme_par_jour_arret = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET'])['NB_VALD'].sum().reset_index()
moyenne_par_arret = données_validation2.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Normalisation des libellés
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.lower().str.strip()

# Somme des valeurs de NB_VALD pour les libellés normalisés
moyenne_par_arret = moyenne_par_arret.groupby('LIBELLE_ARRET')['NB_VALD'].sum().reset_index()

# Suppression des doublons
moyenne_par_arret = moyenne_par_arret.drop_duplicates(subset=['LIBELLE_ARRET'])

print("Nombre de validations par jour par arrêt du 24-07-2022 au 11-08-2022")
display(somme_par_jour_arret)

print("Moyenne (sur la période du 24-07-2022 au 11-08-2022) du nombre de validations par arrêt")
display(moyenne_par_arret)

Nombre de validations par jour par arrêt du 24-07-2022 au 11-08-2022


,JOUR,LIBELLE_ARRET,NB_VALD
0,2022-07-24,ABBESSES,1867
1,2022-07-24,ABLON,600
2,2022-07-24,ACHERES GRAND,39
3,2022-07-24,ACHERES VILLE,3002
4,2022-07-24,AEROPORT CDG1,2955
...,...,...,...
13776,2022-08-11,VOSVES,20
13777,2022-08-11,VULAINES SUR SE,44
13778,2022-08-11,WAGRAM,3063
13779,2022-08-11,YERRES,2729


Moyenne (sur la période du 24-07-2022 au 11-08-2022) du nombre de validations par arrêt


,LIBELLE_ARRET,NB_VALD
0,abbesses,386.886573
1,ablon,172.359310
2,acheres grand,9.893878
3,acheres ville,437.721322
4,aeroport cdg1,629.307638
...,...,...
742,vosves,7.684575
743,vulaines sur se,9.836510
744,wagram,522.766485
745,yerres,452.081411
